In [1]:
import xarray as xr
import rioxarray as rxr
import earthkit.data
from earthkit.transforms import aggregate
from dhis2eo.integrations.pandas import dataframe_to_dhis2_json

In [2]:
from dask.distributed import LocalCluster
cluster = LocalCluster()
client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 96.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51935,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 96.00 GiB
Comm: tcp://127.0.0.1:51947,Total threads: 3
Dashboard: http://127.0.0.1:51951/status,Memory: 24.00 GiB
Nanny: tcp://127.0.0.1:51938,


In [19]:
pm_file = "../data/pm_final_srilanka_linearp.nc"
pm = xr.open_dataset(pm_file, chunks={"time": 100})
pm

<xarray.Dataset> Size: 2GB
Dimensions:                        (time: 1401, lat: 450, lon: 450)
Coordinates:
  * time                           (time) datetime64[ns] 11kB 2020-03-01 ... ...
  * lat                            (lat) float32 2kB 5.505 5.515 ... 9.985 9.995
  * lon                            (lon) float32 2kB 78.0 78.01 ... 82.49 82.5
Data variables:
    __xarray_dataarray_variable__  (time, lat, lon) float64 2GB dask.array<chunksize=(100, 450, 450), meta=np.ndarray>

In [20]:
district_file = "../data/sri-lanka-provinces.geojson"
features = earthkit.data.from_source("file", district_file)
features

,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry
0,Northern Province,LK-4,99731895B93054189817547,LKA,ADM1,"MULTIPOLYGON (((79.91381 8.94183, 79.91835 8.9..."
1,Eastern Province,LK-5,99731895B51072878877669,LKA,ADM1,"POLYGON ((80.75304 8.90515, 80.78104 8.91667, ..."
2,Central Province,LK-2,99731895B28050807675820,LKA,ADM1,"POLYGON ((80.98913 7.72169, 80.98026 7.7161, 8..."
3,North Central Province,LK-7,99731895B66209916164902,LKA,ADM1,"POLYGON ((80.03237 8.52721, 80.0428 8.50005, 8..."
4,North Western Province,LK-6,99731895B5260290378804,LKA,ADM1,"MULTIPOLYGON (((79.77994 8.26209, 79.78165 8.2..."
5,Sabaragamuwa Province,LK-9,99731895B16804022686405,LKA,ADM1,"POLYGON ((80.42215 7.35518, 80.41441 7.35669, ..."
6,Southern Province,LK-3,99731895B60977758614393,LKA,ADM1,"POLYGON ((81.60831 6.58004, 81.60487 6.5688, 8..."
7,Uva Province,LK-8,99731895B77537271836321,LKA,ADM1,"POLYGON ((80.97791 7.62496, 80.97926 7.61174, ..."
8,Western Province,LK-1,99731895B46623695729728,LKA,ADM1,"POLYGON ((79.84202 7.27339, 79.84265 7.26526, ..."


In [21]:
pop_file = "../data/lka_pop_2023_CN_1km_R2025A_UA_v1.tif"
pop = xr.open_dataset(pop_file)
pop_data = pop['band_data']
pop_data = pop_data.rename({"x": "lon", "y": "lat"})
pop_data

<xarray.DataArray 'band_data' (band: 1, lat: 471, lon: 284)> Size: 535kB
[133764 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * lon          (lon) float64 2kB 79.52 79.53 79.54 79.55 ... 81.86 81.87 81.88
  * lat          (lat) float64 4kB 9.838 9.829 9.821 9.813 ... 5.938 5.929 5.921
    spatial_ref  int64 8B ...
Attributes:
    AREA_OR_POINT:  Area
    long_name:      global_pop_2023_CN_1km_R2025A_v1

In [22]:
# pm_day = pm.sel(time='2023-12-01')
pm_data = pm["__xarray_dataarray_variable__"]
pm_data

<xarray.DataArray '__xarray_dataarray_variable__' (time: 1401, lat: 450,
                                                   lon: 450)> Size: 2GB
dask.array<open_dataset-__xarray_dataarray_variable__, shape=(1401, 450, 450), dtype=float64, chunksize=(100, 450, 450), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 11kB 2020-03-01 2020-03-02 ... 2023-12-31
  * lat      (lat) float32 2kB 5.505 5.515 5.525 5.535 ... 9.975 9.985 9.995
  * lon      (lon) float32 2kB 78.0 78.01 78.03 78.04 ... 82.46 82.47 82.49 82.5

In [23]:
pop_aligned = pop_data.interp(lon=pm_data.lon, lat=pm_data.lat, method="nearest")
pop_aligned

<xarray.DataArray 'band_data' (band: 1, lat: 450, lon: 450)> Size: 810kB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B ...
  * lon          (lon) float32 2kB 78.0 78.01 78.03 78.04 ... 82.47 82.49 82.5
  * lat          (lat) float32 2kB 5.505 5.515 5.525 5.535 ... 9.975 9.985 9.995
Attributes:
    AREA_OR_POINT:  Area
    long_name:      global_pop_2023_CN_1km_R2025A_v1

In [24]:
weighted_pm = pm_data * pop_aligned

In [25]:
# pm_data.plot()

In [26]:
# weighted_pm.plot()

In [27]:
# https://www.geopythontutorials.com/notebooks/xarray_zonal_stats.html
# https://earthkit-data.readthedocs.io/en/latest/examples/xarray_engine_chunks_on_dask_cluster.html
# https://github.com/ecmwf/earthkit-transforms/blob/main/src/earthkit/transforms/spatial/_aggregate.py#L378
# https://github.com/ecmwf/earthkit-transforms/blob/main/src/earthkit/transforms/spatial/_aggregate.py#L42

In [28]:
agg_num = aggregate.spatial.reduce(weighted_pm, features, mask_dim="id")
agg_den = aggregate.spatial.reduce(pop_aligned, features, mask_dim="id")

In [13]:
agg_den

<xarray.DataArray 'band_data' (id: 9)> Size: 36B
array([ 177.2038 ,  236.08157,  490.27682,  162.68793,  305.99792,
        403.14877,  461.01895,  207.83444, 1413.1104 ], dtype=float32)
Coordinates:
    band         int64 8B 1
    spatial_ref  int64 8B 0
  * id           (id) int64 72B 0 1 2 3 4 5 6 7 8
Attributes:
    AREA_OR_POINT:  Area
    long_name:      mean global_pop_2023_CN_1km_R2025A_v1
    reduce_dims:    ['lat', 'lon']

In [14]:
pw = agg_num / agg_den
pw.name = "pm25_popweighted"

In [15]:
agg_df = pw.to_dataframe().reset_index()

In [16]:
agg_df

,id,time,band,spatial_ref,pm25_popweighted
0,0,2020-03-01,1,0,39.696083
1,0,2020-03-02,1,0,55.129227
2,0,2020-03-03,1,0,68.584512
3,0,2020-03-04,1,0,44.300611
4,0,2020-03-05,1,0,36.626623
...,...,...,...,...,...
12604,8,2023-12-27,1,0,28.062105
12605,8,2023-12-28,1,0,24.893634
12606,8,2023-12-29,1,0,30.902583
12607,8,2023-12-30,1,0,28.625284


In [17]:
json_dict = dataframe_to_dhis2_json(
    df = agg_df,                                 # aggregated pandas.DataFrame
    org_unit_col = 'id',                         # column containing the org unit id
    period_col = 'time',                         # column containing the period
    value_col = 'pm25_popweighted', # column containing the value
    data_element_id = 'abc123'                   # id of the DHIS2 data element
)

In [18]:
json_dict['dataValues'][:3]

[{'orgUnit': 0,
  'period': '20200301',
  'value': 39.69608288648771,
  'dataElement': 'abc123'},
 {'orgUnit': 0,
  'period': '20200302',
  'value': 55.129226952132775,
  'dataElement': 'abc123'},
 {'orgUnit': 0,
  'period': '20200303',
  'value': 68.58451221887482,
  'dataElement': 'abc123'}]

In [35]:
pm_file = "../data/pm_final_srilanka_linearp.nc"
pm = xr.open_dataset(pm_file, chunks={"time": 100})
pm_data = pm["__xarray_dataarray_variable__"]

pop_file = "../data/lka_pop_2023_CN_1km_R2025A_UA_v1.tif"
pop = xr.open_dataset(pop_file)
pop_data = pop['band_data'].rename({"x": "lon", "y": "lat"})
pop_aligned = pop_data.interp(lon=pm_data.lon, lat=pm_data.lat, method="nearest")

district_file = "../data/sri-lanka-provinces.geojson"
features = earthkit.data.from_source("file", district_file)

agg_num = aggregate.spatial.reduce(weighted_pm, features, mask_dim="id")
agg_den = aggregate.spatial.reduce(pop_aligned, features, mask_dim="id")

pw = agg_num / agg_den
pw.name = "pm25_popweighted"

agg_df = pw.to_dataframe().reset_index()

json_dict = dataframe_to_dhis2_json(
    df = agg_df,                    # aggregated pandas.DataFrame
    org_unit_col = 'id',            # column containing the org unit id
    period_col = 'time',            # column containing the period
    value_col = 'pm25_popweighted', # column containing the value
    data_element_id = 'abc123'      # id of the DHIS2 data element
)

json_dict['dataValues'][:3]

[{'orgUnit': 0,
  'period': '20200301',
  'value': 39.69608288648771,
  'dataElement': 'abc123'},
 {'orgUnit': 0,
  'period': '20200302',
  'value': 55.129226952132775,
  'dataElement': 'abc123'},
 {'orgUnit': 0,
  'period': '20200303',
  'value': 68.58451221887482,
  'dataElement': 'abc123'}]